In [5]:
import numpy as np
from pyomo.environ import *

# ----- PARAMS -----

solver_name = 'glpk'
len_items = 10
capacity = 250
item_max_value = 100
item_max_weight = 100

display_solution = True

# ======= BODY =======

items = range(len_items)
model = ConcreteModel()
model.a = np.random.uniform(0, item_max_value, size=(len(items),))
model.w = np.random.uniform(0, item_max_weight, size=(len(items),))
model.x = Var(items, within=Binary)

def calc_value(model):
    return sum([model.x[i]*model.v[i] for i in items])

def check_weight(model):
    return sum([model.x[i]*model.w[i] for i in items]) <= capacity

model.target = Objective(rule=calc_value, sense=maximize)
model.wlimit = Constraint(rule=check_weight)
opt = SolverFactory(solver_name)
results = opt.solve(model, tee=True)
#model.solutions.load_from(results)
#print(model.solutions)
#print(results)
if display_solution: print(model.x.pprint())

ERROR: Rule failed when generating expression for objective target:
    AttributeError: 'ConcreteModel' object has no attribute 'v'
ERROR: Constructing component 'target' from data=None failed: AttributeError:
    'ConcreteModel' object has no attribute 'v'


AttributeError: 'ConcreteModel' object has no attribute 'v'

In [20]:
[i.value for i in model.x.values()]

[1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 1.0, 0.0]

In [2]:
model.w

NameError: name 'model' is not defined

In [139]:
np.prod(items+1), np.sum(items)

(3628800, 45)

In [137]:
items+1

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [226]:
len_items = 10
items = np.arange(len_items)
model = ConcreteModel()
model.r = np.random.uniform(0, item_max_value, size=(len_items,))
model.p = np.random.uniform(0, item_max_value, size=(len_items,))
model.d = np.random.uniform(0, item_max_value, size=(len_items,))

#x = np.zeros((len_items, len_items)).astype(int)
#for i in range(len_items): x[i,i] = 1
#model.pi = Set(initialize=[items.tolist(), items.tolist()])
#model.ini = Set(items, initialize=items)
model.pi = Var(items, within=Integers, bounds=(0,len_items-1))
for i in items: model.pi[i] = i

#def pi(x):
#    return [np.where(x[i])[0][0] for i in range(len_items)]

def C(model, i):
    t = 0
    ind = model.pi#(model.x)
    r = model.r[ind]
    p = model.p[ind]
    d = model.d[ind]
    for i in items[:i+1]:
        if t < r[i]: t = r[i]
        t += p[i]
    return t

def C_max(model):
    return C(model, len_items-1)

def L(model, i=None):
    ind = model.pi#(model.x)
    r = model.r[ind]
    p = model.p[ind]
    d = model.d[ind]
    if i is None:
        return C_max(model) - d[-1]
    return C(model, i) - d[i]

def L_max(model):
        if len_items == 0: return float('inf')
        return max([L(model, i) for i in items])
    
def check_pi(m):
    #pi = [i.value for i in model.pi.values()]
    return (sorted(m.pi) == items).all()
    
model.obj = Objective(rule=L_max, sense=minimize)

model.sumBound = int(np.sum(items))
model.prodBound = int(np.prod(items+1))

#model.prodConstr = Constraint(expr=int(np.prod(model.pi)) == model.prodBound)
#model.sumConstr = Constraint(rule=lambda m: sum(m.pi) == m.sumBound)

In [227]:
opt = SolverFactory(solver_name)
results = opt.solve(model, tee=True)

    solver failure.
GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --write C:\Users\Admin\AppData\Local\Temp\tmp2hvni1ek.glpk.raw --wglp C:\Users\Admin\AppData\Local\Temp\tmp3ftj3hwh.glpk.glp
 --cpxlp C:\Users\Admin\AppData\Local\Temp\tmp8twbs4kc.pyomo.lp
Reading problem data from 'C:\Users\Admin\AppData\Local\Temp\tmp8twbs4kc.pyomo.lp'...
1 row, 1 column, 1 non-zero
13 lines were read
Writing problem data to 'C:\Users\Admin\AppData\Local\Temp\tmp3ftj3hwh.glpk.glp'...
8 lines were written
GLPK Simplex Optimizer, v4.65
1 row, 1 column, 1 non-zero
Preprocessing...
~     0: obj =   3.888476383e+02  infeas =  0.000e+00
OPTIMAL SOLUTION FOUND BY LP PREPROCESSOR
Time used:   0.0 secs
Memory used: 0.0 Mb (32356 bytes)
Writing basic solution to 'C:\Users\Admin\AppData\Local\Temp\tmp2hvni1ek.glpk.raw'...
11 lines were written


In [225]:
model.pi.pprint()

pi : Size=10, Index=pi_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :     0 :     9 : False :  True : Integers
      1 :     0 :     1 :     9 : False :  True : Integers
      2 :     0 :     2 :     9 : False :  True : Integers
      3 :     0 :     3 :     9 : False :  True : Integers
      4 :     0 :     4 :     9 : False :  True : Integers
      5 :     0 :     5 :     9 : False :  True : Integers
      6 :     0 :     6 :     9 : False :  True : Integers
      7 :     0 :     7 :     9 : False :  True : Integers
      8 :     0 :     8 :     9 : False :  True : Integers
      9 :     0 :     9 :     9 : False :  True : Integers


In [200]:
p = 1
for i in model.pi: p *= (i+1)

In [212]:
sum(model.pi)

45

In [192]:
model.pi.pprint()

pi : Size=10, Index=pi_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :     0 :     9 : False : False : Integers
      1 :     0 :     1 :     9 : False : False : Integers
      2 :     0 :     2 :     9 : False : False : Integers
      3 :     0 :     3 :     9 : False : False : Integers
      4 :     0 :     4 :     9 : False : False : Integers
      5 :     0 :     5 :     9 : False : False : Integers
      6 :     0 :     6 :     9 : False : False : Integers
      7 :     0 :     7 :     9 : False : False : Integers
      8 :     0 :     8 :     9 : False : False : Integers
      9 :     0 :     9 :     9 : False : False : Integers


In [161]:
model.pi.pprint()

pi : Size=10, Index=pi_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     0 :  None :     9 : False :  True : Integers
      1 :     0 :  None :     9 : False :  True : Integers
      2 :     0 :  None :     9 : False :  True : Integers
      3 :     0 :  None :     9 : False :  True : Integers
      4 :     0 :  None :     9 : False :  True : Integers
      5 :     0 :  None :     9 : False :  True : Integers
      6 :     0 :  None :     9 : False :  True : Integers
      7 :     0 :  None :     9 : False :  True : Integers
      8 :     0 :  None :     9 : False :  True : Integers
      9 :     0 :  None :     9 : False :  True : Integers


In [141]:
model.pi.pprint()

pi : Size=10, Index=pi_index
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :     1 :  None :    10 : False :  True : PositiveIntegers
      1 :     1 :  None :    10 : False :  True : PositiveIntegers
      2 :     1 :  None :    10 : False :  True : PositiveIntegers
      3 :     1 :  None :    10 : False :  True : PositiveIntegers
      4 :     1 :  None :    10 : False :  True : PositiveIntegers
      5 :     1 :  None :    10 : False :  True : PositiveIntegers
      6 :     1 :  None :    10 : False :  True : PositiveIntegers
      7 :     1 :  None :    10 : False :  True : PositiveIntegers
      8 :     1 :  None :    10 : False :  True : PositiveIntegers
      9 :     1 :  None :    10 : False :  True : PositiveIntegers


In [131]:
model.r, model.p, model.d

(array([15.84676976, 61.22217926, 30.47524542,  6.47262513, 48.42043118,
        97.79381601, 16.44531101, 35.27231539, 71.27209535, 37.25228873]),
 array([95.44436515, 87.85490063, 46.98090899, 92.02289949, 71.55950788,
         0.42151054, 45.55336097, 30.45597344, 34.57465577, 20.50433244]),
 array([83.3569799 , 78.76384817, 78.41697985, 87.76046751, 42.18376495,
        19.02875812, 63.52032455, 19.82163165, 27.08161342, 51.33987531]))

In [132]:
L_max(model), L(model, i=len_items-1)

(493.6332392013347, 489.8793097567972)

In [79]:
x = np.zeros((len_items, len_items))
for i in range(len_items): x[i,i] = 1

In [115]:
model.x

In [82]:
ind = [np.where(x[i])[0][0] for i in range(len_items)]

In [85]:
model.r[ind]

array([92.8245235 , 27.76309551, 34.66435279, 63.88032294, 29.60103406,
       58.32664599,  0.98691444,  8.38265321, 57.3448407 , 17.67753785])

In [92]:
[[i for i in range(len_items)] for j in range(len_items)]

[[0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]]

In [124]:
x.flatten().tolist()

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1]